In [ ]:
%%time
############Crop the 384-array image into individual color blocks for each mutant.
#The cutting step is not mandatory, but it facilitates the subsequent review of the actual images corresponding to the identified mutants. Therefore, we perform this cutting step.
import os 
import cv2
import numpy as np
import pylab
import matplotlib.pyplot as plt

def file_name2(file_dir): 
    L=[] 
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if os.path.splitext(file)[1] == '.png':
                L.append(os.path.join(root, file))
    return L

def cv_imread(filePath):#
    cv_img=cv2.imdecode(np.fromfile(filePath,dtype=np.uint8),-1)
    return cv_img 

def divide_spot(file_name):
    img = cv_imread(file_name)
    file_name=file_name.split(".png")[0].replace("−","-")
    os.mkdir(file_name) 
    #print(":",img.shape)
    w=img.shape[1]
    h=img.shape[0]
    h0=h/16
    w0=w/24   
    j=0
    while j<16:
        i=0
        while i<24:
            img_names=file_name+"/"+str(24*j+i+1)+".png"
            a=round(j*h0)##Rounding may sometimes result in a loss of 1 pixel.
            b=round((j+1)*h0)
            c=round(i*w0)
            d=round((i+1)*w0)
            img0=img[a:b,c:d]
            cv2.imencode('.png', img0)[1].tofile(img_names)
            i+=1
        j+=1
    


for i in file_name2('path_to_the_images'):#
    if "the Special characters used for identifying target images"in i:#############
        print(i)
        divide_spot(i)

In [ ]:
%%time
##############Store the values of regions that fall within the RGB range and save them in HSV format.###########
import os 
import cv2
import numpy as np
import pylab
import matplotlib.pyplot as plt

def file_name1(file_dir): 
    F=[] 
    for root, dirs, files in os.walk(file_dir):
        for foldername in dirs:
            if "the Special characters used for identifying target" in foldername:###########
                F.append(os.path.join(root, foldername))
    return F

def file_name2(file_dir): 
    L=[] 
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if os.path.splitext(file)[1] == '.png':
                L.append(os.path.join(root, file))
    return L

def cv_imread(filePath):#
    cv_img=cv2.imdecode(np.fromfile(filePath,dtype=np.uint8),-1)
    return cv_img 
 

def convert_hsv(file_name,f1): 
    imgy = cv_imread(file_name)
    img=cv2.cvtColor(imgy,cv2.COLOR_BGR2HSV)#HSV
    #gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #img=gray.reshape((-1))

    m=img.shape[0]
    n=img.shape[1]
    m1=0
    while m1<m:
        n1=0
        while n1<n:

            if len(img[m1,n1])==3:
                (b,g,r) = img[m1,n1]#
            elif len(img[m1,n1])==4:
                (b,g,r,h) = img[m1,n1]
   

            if ((b<10 or g<10 or r<10) and (int(b)+int(g)+int(r))<70) or ((b<6 or g<6 or r<6) and (int(b)+int(g)+int(r))<75):
                flag=0
           
            else:

                f1.write(file_name.split("/")[-4]+"__"+file_name.split("tt/")[-1].replace(",",";")+","+str(m1)+","+str(n1)+","+str(b)+","+str(g)+","+str(r)+"\n")
            n1+=1
        m1+=1


def change_black(image):#
    img = cv_imread(image)
    m=img.shape[0]
    n=img.shape[1]
    m1=0
    while m1<m:
        n1=0
        while n1<n:
            (b,g,r,h) = img[m1,n1]#
            if b<190 or g<190 or r<190:#
                
                b=0
                g=0
                r=0
                img[m1,n1]=(b,g,r,h)
            n1+=1
        m1+=1
    return img          
        
        

f1=open('pam_fhsv.csv','w') 
f1.write("name"+",m1,n1,"+"H"+","+"S"+","+"V"+"\n")
for j in file_name1('The parent directory that contains all the image cropping folders.'):
    print (j)
    if "the Special characters used for identifying target images" in j:
        #print (j)
        for i in file_name2(j):
            convert_hsv(i,f1,f2)
        
        #################
            #template = change_black(i)
            #new_names=i.split(".png")[0]+"_190.png"
            #cv2.imencode('.png', template)[1].tofile(new_names)
        
f1.close()


In [ ]:
%%time
import numpy as np
from scipy import stats
########################
f1=open('pam_fhsv.csv','r')
f2=open('pamnew_fhsvuniqallp.csv','w')


dictseq={}
a=f1.readline().strip()
a=f1.readline().strip()
while 1:
    a=f1.readline().strip()
    if a=="":break
    b=a.split(",")
    name=str(b[0])
    h=int(b[3])
    s=int(b[4])
    v=int(b[5])
    if name in dictseq:
        dictseq[name][0].append(h)
        dictseq[name][1].append(s)
        dictseq[name][2].append(v)
    else:
        dictseq[name]=[[h],[s],[v]]
    
print(len(dictseq))
f2.write("name,num1,num2,num3,modeh,modes,modev,meanh,means,meanv,medianh,medians,medianv,allh,alls,allv\n")
for key in dictseq:
    f2.write(str(key)+","+str(len(dictseq[key][0]))+","+str(len(dictseq[key][1]))+","+str(len(dictseq[key][2]))+
             ","+str(stats.mode(dictseq[key][0])[0][0])+","+str(stats.mode(dictseq[key][1])[0][0])+","+str(stats.mode(dictseq[key][2])[0][0])+
             ","+str(np.mean(dictseq[key][0]))+","+str(np.mean(dictseq[key][1]))+","+str(np.mean(dictseq[key][2]))+
             ","+str(np.median(dictseq[key][0]))+","+str(np.median(dictseq[key][1]))+","+str(np.median(dictseq[key][2]))+
             ","+str(np.sum(dictseq[key][0]))+","+str(np.sum(dictseq[key][1]))+","+str(np.sum(dictseq[key][2]))+"\n")

f1.close()
f2.close()

print("The end!")

In [ ]:
%%time
import numpy as np
from scipy import stats
#########################
f2=open('pamnew_fhsvuniqalluniq.csv','w')
f1=open('pamnew_fhsvuniqall.csv','r')
a=f1.readline().strip()
f2.write("name,num,sumall,value\n")
while 1:
    a=f1.readline().strip()
    if a=="":break
    b=a.split(",")
    c=float(b[1])
    d=float(b[7])
    e=float(b[10])
    f=float(b[12])
    v=(e+2.8816)/161.25
    vv=round(v,3)
    f2.write(b[0]+","+b[1]+","+b[-1]+","+str(vv)+"\n")

f1.close()
f2.close()

print("The end!")

In [ ]:
%%time
import numpy as np
from scipy import stats

f2=open('pamnew_fhsvuniqname.csv','w')
f1=open('pamnew_fhsvuniqalluniq.csv','r')
############################
def changeseq_384(num):
    a=(num-1)//24
    tail=(num-1)%24+1
    head=chr(65+a)
    body=head+str(tail)
    return body

while 1:
    a=f1.readline().strip()
    if a=="":break
    if "name" in a:
        f2.write(a+"\n")
    else:
        try:
            
            b=int(a.split(".png")[0].split("\\")[-1])
            c=a.split("\\")[0]+"-"+changeseq_384(b)
            d=a.split(",")
            f2.write(c+","+','.join(d[1:])+"\n")
        except:
            print(a)
            b=int(a.split(".png")[0].split("\\")[-1])
            print("1")
            c=a.split("\\")[0]+"-"+changeseq_384(b)
            d=a.split(",")
            print("2")
            print(b)
            print(c)
            print(d[1])
            f2.write(c+","+','.join(d[1:])+"\n")

f1.close()
f2.close()

print("The end!")

In [ ]:
%%time

import numpy as np
from scipy import stats

#f1=open('g_fhsvuniqname.csv','r')
#f3=open('g96_fhsvuniqname.csv','r')
f3=open('pamnew_fhsvuniqname.csv','r')
f2=open('pamnew.csv','w')

def changeseq_384(num):
    a=(num-1)//24
    tail=(num-1)%24+1
    head=chr(65+a)
    body=head+str(tail)
    return body
def dictseq_0(dictseq):
    i=1
    while i<385:
        dictseq[changeseq_384(i)]=changeseq_384(i)+",NA"
        i+=1
    return dictseq

    

dictseq={}
b=f3.readline()
f2.write("plate,"+b)

while 1:
    a=f3.readline().strip()
    if a=="":break
    b=a.split(",")   
    if len(b[0].split("-"))==3:
    
        name=b[0].split("-")[0]+"-"+b[0].split("-")[1]
    elif len(b[0].split("-"))==2:
        name=b[0].split("-")[0]
    else:print(a)
    site=b[0].split("-")[-1]
    if name in dictseq:
        dictseq[name][site]=a
    else:
        dictseq1={}
        dictseq[name]=dictseq_0(dictseq1)
        dictseq[name][site]=a
for key in sorted(dictseq) :
    for keys in dictseq[key]:
        if "NA" in dictseq[key][keys]:
            f2.write(key+","+key+"-"+dictseq[key][keys]+",NA,NA\n")
        else:
            f2.write(key+","+dictseq[key][keys].replace(" ","-")+"\n")


        

#f1.close()
f2.close()
f3.close()
print("The end!")